In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


In [2]:
import matplotlib.pyplot as plt
import plotly.express as px

In [3]:
oil = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/oil.csv")
holidays = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv")
stores = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/stores.csv")
train = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/train.csv")
test = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/test.csv")
transactions = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/transactions.csv")

# EDA

Question: Is there anything wrong with me using 'dcoilwtico_interpolate' for my predictions? Anything to be worries about?

In [4]:
print(f"{oil.isna()['dcoilwtico'].sum()} Missing Values in dcoilwtico")
oil['dcoilwtico_interpolate'] = oil.interpolate(method='linear')['dcoilwtico']
display(oil.head())
px.line(oil, x='date',y='dcoilwtico_interpolate')

43 Missing Values in dcoilwtico


/tmp/ipykernel_17/4242653037.py:2: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  oil['dcoilwtico_interpolate'] = oil.interpolate(method='linear')['dcoilwtico']


,date,dcoilwtico,dcoilwtico_interpolate
0,2013-01-01,NaN,NaN
1,2013-01-02,93.14,93.14
2,2013-01-03,92.97,92.97
3,2013-01-04,93.12,93.12
4,2013-01-07,93.20,93.20


Analysis
* Trends: There is a downward trend in dcoilwtico over time.
* Seasonality: Doesn't appear to be any seasonal trend in the data.
    * Might have to look at different granularity
* Cycles: dcoilwtico appears to change in cycles. Rises and falls are not of fixed frequency.

In [5]:
display(transactions.isna().sum())
display(transactions.head())
px.line(transactions, x='date', y='transactions', color = 'store_nbr')

date            0
store_nbr       0
transactions    0
dtype: int64

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


Analysis
* Trends: No trend. Transactions relatively constant over time.
* Seasonality: Seems to be some seasonality with transactions being generally higher around the summer.
    * Modify granularity to see if seasonality exists in different intervals. 
* Cycles: No obvious cyclic behavior since ups and downs seem to be associated with time
    * Unless I misread cycles as seasons

In [6]:
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [7]:
# Why is this sum the same as mean?
train.groupby(['family', 'date', 'store_nbr']).sales.sum().reset_index()

,family,date,store_nbr,sales
0,AUTOMOTIVE,2013-01-01,1,0.000
1,AUTOMOTIVE,2013-01-01,2,0.000
2,AUTOMOTIVE,2013-01-01,3,0.000
3,AUTOMOTIVE,2013-01-01,4,0.000
4,AUTOMOTIVE,2013-01-01,5,0.000
...,...,...,...,...
3000883,SEAFOOD,2017-08-15,50,15.314
3000884,SEAFOOD,2017-08-15,51,52.876
3000885,SEAFOOD,2017-08-15,52,7.000
3000886,SEAFOOD,2017-08-15,53,5.000


In [8]:
# Plot sales for each family with color as store

In [9]:
train.family.unique()

array(['AUTOMOTIVE', 'BABY CARE', 'BEAUTY', 'BEVERAGES', 'BOOKS',
       'BREAD/BAKERY', 'CELEBRATION', 'CLEANING', 'DAIRY', 'DELI', 'EGGS',
       'FROZEN FOODS', 'GROCERY I', 'GROCERY II', 'HARDWARE',
       'HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES',
       'HOME CARE', 'LADIESWEAR', 'LAWN AND GARDEN', 'LINGERIE',
       'LIQUOR,WINE,BEER', 'MAGAZINES', 'MEATS', 'PERSONAL CARE',
       'PET SUPPLIES', 'PLAYERS AND ELECTRONICS', 'POULTRY',
       'PREPARED FOODS', 'PRODUCE', 'SCHOOL AND OFFICE SUPPLIES',
       'SEAFOOD'], dtype=object)

In [10]:
train

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0
1,1,2013-01-01,1,BABY CARE,0.000,0
2,2,2013-01-01,1,BEAUTY,0.000,0
3,3,2013-01-01,1,BEVERAGES,0.000,0
4,4,2013-01-01,1,BOOKS,0.000,0
...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8


In [11]:
sales_fam = train.groupby(["family", "date"]).sales.sum().reset_index()
sales_fam = sales_fam.pivot(index='date', columns='family', values='sales')
sales_fam

family,AUTOMOTIVE,BABY CARE,BEAUTY,BEVERAGES,BOOKS,BREAD/BAKERY,CELEBRATION,CLEANING,DAIRY,DELI,...,MAGAZINES,MEATS,PERSONAL CARE,PET SUPPLIES,PLAYERS AND ELECTRONICS,POULTRY,PREPARED FOODS,PRODUCE,SCHOOL AND OFFICE SUPPLIES,SEAFOOD
date,,,,,,,,,,,,,,,,,,,,,
2013-01-01,0.0,0.0,2.0,810.0,0.0,180.58900,0.0,186.0,143.0,71.090000,...,0.0,110.801000,25.0,0.0,0.0,42.637000,37.847000,0.00000,0.0,0.000000
2013-01-02,255.0,0.0,207.0,72092.0,0.0,26246.31900,0.0,74629.0,23381.0,15754.500000,...,0.0,20871.464028,17204.0,0.0,0.0,13975.884938,5338.111976,0.00000,0.0,1526.750002
2013-01-03,161.0,0.0,125.0,52105.0,0.0,18456.48002,0.0,55893.0,18001.0,11172.455000,...,0.0,16597.398113,12568.0,0.0,0.0,10674.393983,3591.388005,0.00000,0.0,1094.310994
2013-01-04,169.0,0.0,133.0,54167.0,0.0,16721.96901,0.0,52064.0,18148.0,10143.209000,...,0.0,21625.963055,11303.0,0.0,0.0,10772.515038,4472.965990,0.00000,0.0,1293.120995
2013-01-05,342.0,0.0,191.0,77818.0,0.0,22367.76108,0.0,70128.0,23082.0,13734.945010,...,0.0,20879.091050,16819.0,0.0,0.0,13475.009055,5830.073020,0.00000,0.0,1245.637004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-08-11,441.0,7.0,351.0,189111.0,0.0,28204.48600,870.0,55682.0,44909.0,18680.488004,...,500.0,26944.092010,15978.0,587.0,654.0,25318.297990,5199.494021,118738.14300,3523.0,1272.615997
2017-08-12,403.0,3.0,369.0,182318.0,0.0,27403.54800,849.0,57935.0,42854.0,15689.893010,...,483.0,17775.650112,15903.0,508.0,712.0,19134.510058,4573.465992,111788.35090,3644.0,1028.030006
2017-08-13,481.0,9.0,433.0,202354.0,1.0,33262.75700,544.0,61757.0,50112.0,17593.274020,...,469.0,18633.015039,18188.0,541.0,741.0,20509.265004,4941.509018,125923.80240,3718.0,1118.047000


In [12]:
# from plotly.subplots import make_subplots
# import plotly.graph_objects as go


# num_families = len(train.family.unique())

# # Initialize figure with subplots
# fig = make_subplots(
#     rows= num_families // 3 + 1, cols= 3
# )

# print("Shape", num_families // 3 + 1, 3)

# n = 0
# for category in train.family.unique():    
#     row = n // 3 + 1
#     column = n % 3 + 1
    
#     fig.add_trace(go.Scatter(x=sales_fam.index, y=sales_fam[category]), row=n, col=column)

#     print(n, column)
    
#     n += 1

In [13]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Assuming train.family and sales_fam are properly defined
num_families = len(train.family.unique())

# Initialize figure with subplots
fig = make_subplots(
    rows=num_families // 3 + 1, cols=3
)

n = 0  # Start from zero
for category in train.family.unique():
    row = n // 3 + 1  # Use integer division for row calculation
    column = n % 3 + 1  # Modulo to calculate column position
    
    # Add trace for each family category
    fig.add_trace(go.Scatter(x=sales_fam.index, y=sales_fam[category], name=category), row=row, col=column)
    
    n += 1

# Show the figure
fig.show()


In [14]:
holidays.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [15]:
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4
